# Attention

NB: Puisque le projet se fait sur Google Colab, nous devons utiliser Drive pour stocker les fichiers de façon durable.

Si vous exécuter le code localement, vous n'avez pas besoin de ce bloc de code. Il faudra penser à mettre à jour la variable root_path deux blocs plus loin.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# On s'assure que vous avez exactement les mêmes versions que les librairies utilisées dans ce notebook
!pip install -q pandas==2.2.2 scikit-learn==1.5.2 matplotlib==3.8.0 numpy==1.26.4 tqdm==4.66.6 imbalanced-learn==0.12.4 shap==0.46.0 equipy==0.0.6a0.dev0

# Prédictions

Le but de ce notebook est d'illustrer la partie prédiction. Nous avons déjà entraîné un modèle dans le notebook Modélisation, et pour plus de visibilité et vous épargner sa lecture, nous avons exporté le meilleur modèle avec la meilleure approche, pour l'appliquer ici sur le jeu de test et l'exporter comme convenu.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

random_state=24

root_path = "/content/drive/My Drive/Projet CNAM"
path_test = f"{root_path}/test.csv"

df_test = pd.read_csv(path_test, sep=",")
df_test.shape

(30000, 11)

Le code nécessite de réutiliser le même code de preprocessing défini dans la partie Modélisation et réuni dans une pipeline.

In [ ]:
import numpy as np

def preprocess_quali(df):
    feats_quali=[]
    # on s'occupe d'abord des variables binaires
    # preference : valeur = 1 <=> corrélation positive avec cible
    df["feat_permis"]=(df["permisConduire"]==1).astype(int)
    feats_quali.append("feat_permis")
    df["feat_sans_assurance"]=(df["dejaAssure"]==0).astype(int)
    feats_quali.append("feat_sans_assurance")
    df["feat_accident"]=(df["accidentVehicule"]=="Yes").astype(int)
    feats_quali.append("feat_accident")
    df["feat_homme"]=(df["genre"]=="Male").astype(int)
    feats_quali.append("feat_homme")
    # on traite ensuite la valeur multicatégorielle (ageVehicle)
    df["feat_age_vehicule>1"]=(df.ageVehicule.isin(["1-2 Year", "> 2 Years"])).astype(int)
    feats_quali.append("feat_age_vehicule>1")
    return df, feats_quali

def truncate_quantile(series, quantile=0.95):
    q95=int(series.quantile(quantile))
    series_output=series.copy()
    series_output[series_output>q95]=q95+1
    return series_output

def preprocess_quanti(df, norm_dict={}):
    feats_quanti=[]
    # on corrige d'abord la prime annuelle
    two_trailing_zeroes=(df["primeAnnuelle"].astype(str).str[-2:]=="00")
    big_premium=(df["primeAnnuelle"]>50000)
    df["feat_prime"]=df["primeAnnuelle"]
    df.loc[big_premium & two_trailing_zeroes,"feat_prime"]=df.loc[big_premium & two_trailing_zeroes,"primeAnnuelle"]/100
    feats_quanti.append("feat_prime")
    # on corrige ensuite les âges
    df["feat_age"]=truncate_quantile(df["age"])
    df["feat_age_q95"]=(df["feat_age"]==df["feat_age"].max()).astype(int)
    feats_quanti.append("feat_age")
    # on corrige enfin le temps assuré
    df["feat_temps_assure"]=truncate_quantile(df["tempsAssure"])
    df["feat_temps_assure_q95"]=(df["feat_temps_assure"]==df["feat_temps_assure"].max()).astype(int)
    feats_quanti.append("feat_temps_assure")
    # on normalise: si on n'a pas de dictionnaire pour normaliser, on en crée un et on l'utilise
    if not norm_dict:
        for col in feats_quanti:
            norm_dict[col]={"min":df[col].min(), "max":df[col].max()}
    for col in feats_quanti:
        df[col]=(df[col]-norm_dict[col]["min"])/(norm_dict[col]["max"]-norm_dict[col]["min"])
        # pour garder la valeur dans le périmètre dans le cas du jeu de test et d'eval
        df.loc[df[col]>1,col]=1
        df.loc[df[col]<0,col]=0
    feats_quanti.extend(["feat_age_q95", "feat_temps_assure_q95"])
    return df, norm_dict, feats_quanti

def process_other(df, dict_cat={}):
    no_dict_cat=False
    feats_other=[]
    target_prop=0
    if not dict_cat:
        no_dict_cat=True
        target_prop=df[target].mean()
    list_other=['codeRegion', 'canalDistribution']
    for col in list_other:
        if no_dict_cat:
            dict_cat[col]={}
            # identification des classes majoritaires
            df_proportions=pd.DataFrame(df[col].value_counts(normalize=True)).reset_index(drop=False)
            # on identifie le "coude" comme étant l'instant du plus grand écart absolu d'une catégorie à la suivante
            df_proportions["diff"]=df_proportions.proportion.diff().shift(-1).abs()
            max_prop=df_proportions.loc[df_proportions["diff"]==df_proportions["diff"].max(),"proportion"].iloc[0]
            # les classes majoritaires sont celles à gauche du coude
            dict_cat[col]["list_maj"]=df_proportions.loc[df_proportions.proportion>=max_prop, col].tolist()
            # identification des classes minoritaires positives
            df_other=df[~df[col].isin(dict_cat[col]["list_maj"])]
            df_crosstab=pd.crosstab(df_other[target], df_other[col], normalize="columns").transpose().reset_index()
            # une classe est minoritaire positive si sa proportion de valeurs positives > proportion moyenne du dataset
            dict_cat[col]["list_min_pos"]=df_crosstab.loc[df_crosstab[1]>=target_prop, col].tolist()
            dict_cat[col]["list_min_neg"]=df_crosstab.loc[df_crosstab[1]<target_prop, col].tolist() # respectivement, minoritaire négative
        #application : on initialise la colonne à l'identique pour maintenir les classes majoritaires
        df["feat_"+col]=df[col].astype(str)
        # si une classe appartient aux classes minoritaires (positives ou négatives), elle reçoit le label correspondant
        df.loc[df[col].isin(dict_cat[col]["list_min_pos"]), "feat_"+col]=f"pos"
        df.loc[df[col].isin(dict_cat[col]["list_min_neg"]), "feat_"+col]=f"neg"
        # si une classe n'est dans aucune catégorie, elle reçoit le label "autre"
        df.loc[~df[col].isin(dict_cat[col]["list_maj"]+dict_cat[col]["list_min_pos"]+dict_cat[col]["list_min_neg"]), "feat_"+col]=f"other"
        # on applique un one hot encoding personnalisé pour tenir compte du fait qu'on ne veut pas de "other"
        # on drop également les colonnes minoritaires négatives + other pour éviter les multicolinéarités
        for i in [i for i in df[f"feat_{col}"].unique() if i not in [f"other_{col}", f"neg_{col}"]]:
            df[f"feat_{col}_{i}"]=(df[f"feat_{col}"]==i).astype(int)
            feats_other.append(f"feat_{col}_{i}")

    return df, dict_cat, feats_other

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class Preprocessor(BaseEstimator, TransformerMixin):
# la classe doit avoir des fonctions que sklearn connaît
# telles que fit, transform et fit_transform
# sklearn la traitera donc comme un modèle comme les autres
  def __init__(self, list_feat):
    self.norm_dict = None
    self.dict_cat = None
    self.list_feat = list_feat
    return None

  def fit(self, X, y=None):
    self.fit_transform(X)
    return self

  def fit_transform(self, X, y=0):
    # fit_transform gère le jeu de train et initialise les valeurs
    X, _=preprocess_quali(X.copy())
    X, self.norm_dict, _=preprocess_quanti(X)
    X, self.dict_cat, _=process_other(X)
    return X[self.list_feat]

  def transform(self, X):
    # transform sert à traiter le jeu de test avec les valeurs déjà initialisées
    # grâce au jeu de train
    X, _=preprocess_quali(X.copy())
    if self.norm_dict:
        X, _, _=preprocess_quanti(X, self.norm_dict)
    else:
        print("no norm_dict initialized")
        raise ValueError
    if self.dict_cat:
        X, _, _=process_other(X, self.dict_cat)
    else:
        print("no dict_cat initialized")
        raise ValueError
    return X[self.list_feat]

Nous allons donc lire le pickle exporté dans l'autre notebook, et l'appliquer sur le jeu de test pour avoir les prédictions et les probabilités

Le modèle inclut déjà le preprocessing et l'application d'un seuil personnalisé

In [ ]:
import pickle
filename = f"{root_path}/best_model.pkl"

model_card = pickle.load(open(filename, 'rb'))

list_feat=model_card["list_feat"]
target=model_card["target"]
model=model_card["model"]

In [ ]:
df_test["interesse"]=model.predict(df_test)
df_test["probability"]=model.predict_proba(df_test)[:, 1]

In [18]:
path_output = f"{root_path}/prediction_11.csv"

df_test[["id", "probability", "interesse"]].to_csv(path_output, sep=",")